In [3]:
# Author : Ravi Ranjan
# IIT Bombay 2018-2020

import json
import numpy as np

In [4]:
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [5]:
with open('val_sceneGraphs.json') as f:
    gqa_vg_train = json.load(f)

In [6]:
num_images = 0
for idx, record in gqa_vg_train.items():
    num_images += 1
num_images

10696

In [7]:
temp_dict = {}
# number of images in the catalog = N
N = 100
i = 0
for idx, record in gqa_vg_train.items():
    temp_dict[idx] = record
    i += 1
    if i == N:
        break
gqa_vg_train = temp_dict
len(gqa_vg_train)

100

In [8]:
# object vocab creation :

In [9]:
redundant_obj_dict = {}
for idx, record in gqa_vg_train.items():
    for obj_key, obj in record['objects'].items():
        redundant_obj_dict[obj_key] = obj['name']


In [10]:
len(redundant_obj_dict)

1580

In [11]:
n_items = take(1, redundant_obj_dict.items())
print(n_items)

[('2716708', 'nose')]


In [12]:
inv_redundant_obj_dict = {v: k for k, v in redundant_obj_dict.items()}
len(inv_redundant_obj_dict)

422

In [13]:
# of unique objects : 
print(len(inv_redundant_obj_dict))

422


In [14]:
n_items = take(1, inv_redundant_obj_dict.items())
print(n_items)

[('nose', '3952616')]


In [15]:
object_vocab = {}
index = 0
for key in inv_redundant_obj_dict:
    object_vocab[key] = index
    index += 1

In [16]:
n_items = take(1, object_vocab.items())
print(n_items)

[('nose', 0)]


In [17]:
len(object_vocab)

422

In [18]:
# imageid vocab creation
image_vocab = {}
index = 0
for idx, record in gqa_vg_train.items():
    image_vocab[idx] = index
    index += 1

In [19]:
n_items = take(1, image_vocab.items())
print(n_items)

[('2353884', 0)]


In [20]:
# creation of relationship vocab
relation_vocab = {}
index  = 0
for idx, record in gqa_vg_train.items():
    for obj_key, obj in record['objects'].items():
        for relation in obj['relations']:
            if relation['name'] not in relation_vocab:
                relation_vocab[relation['name']] = index
                index += 1
len(relation_vocab)  

61

In [21]:
# creating dummy relation for object_attr relationship
dummy_rel = 'dummy'
print(len(relation_vocab))
relation_vocab['dummy'] = len(relation_vocab) + 1
print(len(relation_vocab))

61
62


In [22]:
n_items = take(61, relation_vocab.items())
print(n_items)

[('of', 0), ('to the right of', 1), ('to the left of', 2), ('wearing', 3), ('reading', 4), ('holding', 5), ('on', 6), ('in', 7), ('under', 8), ('below', 9), ('with', 10), ('on top of', 11), ('above', 12), ('in front of', 13), ('behind', 14), ('by', 15), ('riding', 16), ('playing with', 17), ('parked next to', 18), ('parked in front of', 19), ('near', 20), ('next to', 21), ('mounted on', 22), ('hanging from', 23), ('lying in', 24), ('lying on', 25), ('walking on', 26), ('walking with', 27), ('sitting at', 28), ('using', 29), ('watching', 30), ('inside', 31), ('standing by', 32), ('hanging on', 33), ('filled with', 34), ('full of', 35), ('coming from', 36), ('standing on', 37), ('around', 38), ('touching', 39), ('covered with', 40), ('sitting on', 41), ('walking in', 42), ('stacked on', 43), ('parked along', 44), ('sitting near', 45), ('parked near', 46), ('parked on', 47), ('flying over', 48), ('at', 49), ('covering', 50), ('covered by', 51), ('mounted to', 52), ('throwing', 53), ('play

In [23]:
# attribute vocab creation
# creation of inverted attribute vocab
attr_vocab = {}
index = 0
for idx, record in gqa_vg_train.items():
    for obj_key, obj in record['objects'].items():
        for attr in obj['attributes']:
            if attr not in attr_vocab:
                attr_vocab[attr] = index
                index += 1
len(attr_vocab)

166

In [24]:
n_items = take(50, relation_vocab.items())
print(n_items)

[('of', 0), ('to the right of', 1), ('to the left of', 2), ('wearing', 3), ('reading', 4), ('holding', 5), ('on', 6), ('in', 7), ('under', 8), ('below', 9), ('with', 10), ('on top of', 11), ('above', 12), ('in front of', 13), ('behind', 14), ('by', 15), ('riding', 16), ('playing with', 17), ('parked next to', 18), ('parked in front of', 19), ('near', 20), ('next to', 21), ('mounted on', 22), ('hanging from', 23), ('lying in', 24), ('lying on', 25), ('walking on', 26), ('walking with', 27), ('sitting at', 28), ('using', 29), ('watching', 30), ('inside', 31), ('standing by', 32), ('hanging on', 33), ('filled with', 34), ('full of', 35), ('coming from', 36), ('standing on', 37), ('around', 38), ('touching', 39), ('covered with', 40), ('sitting on', 41), ('walking in', 42), ('stacked on', 43), ('parked along', 44), ('sitting near', 45), ('parked near', 46), ('parked on', 47), ('flying over', 48), ('at', 49)]


In [25]:
print("done with vocab creations")

done with vocab creations


In [26]:
master_ds_obj_rel_obj_imageid = {}
for idx, record in gqa_vg_train.items():
    for obj_key, obj in record['objects'].items():
        for relation in obj['relations']:
            obj_rel_obj = (obj['name'], relation['name'], redundant_obj_dict[relation['object']])
            tuple_indices = []
            if obj_rel_obj in master_ds_obj_rel_obj_imageid:
                master_ds_obj_rel_obj_imageid[obj_rel_obj].extend([idx])
            else:
                master_ds_obj_rel_obj_imageid[obj_rel_obj] = [idx]


In [27]:
# making the obj_rel_obj_imageid have unique images in its list (removing the redundancy of same tuple apprearing more than once in the same image)
for key in master_ds_obj_rel_obj_imageid:
    unique_image_list = []
    for item in master_ds_obj_rel_obj_imageid[key]:
        unique_image_list.append(item)
    unique_image_list = list(set(unique_image_list))
    master_ds_obj_rel_obj_imageid[key] = unique_image_list

In [28]:
n_items = take(20, master_ds_obj_rel_obj_imageid.items())
print(n_items)

[(('nose', 'of', 'man'), ['2353884']), (('nose', 'to the right of', 'roof'), ['2353884']), (('nose', 'to the left of', 'eyes'), ['2353884']), (('eyes', 'to the right of', 'roof'), ['2353884']), (('eyes', 'to the right of', 'nose'), ['2353884']), (('eyes', 'of', 'man'), ['2353884']), (('sign', 'to the right of', 'man'), ['2317105', '2353884', '2368884']), (('ear', 'of', 'man'), ['2353884']), (('ear', 'to the right of', 'roof'), ['2353884']), (('socks', 'to the left of', 'shorts'), ['2353884']), (('man', 'wearing', 'socks'), ['2353884']), (('man', 'wearing', 'shorts'), ['2360094', '2353884', '2380479', '2374892']), (('man', 'to the left of', 'sign'), ['2317105', '2353884', '2368884']), (('man', 'to the right of', 'man'), ['2360094', '2353884', '2387230', '2413380']), (('man', 'to the right of', 'frisbee'), ['2353884']), (('woman', 'to the left of', 'shorts'), ['2353884']), (('shorts', 'to the right of', 'shoe'), ['2353884']), (('shorts', 'to the right of', 'woman'), ['2353884']), (('shor

In [29]:
master_ds_obj_rel_attr_imageid = {}
for idx, record in gqa_vg_train.items():
    for obj_key, obj in record['objects'].items():
        for attribute in obj['attributes']:
            obj_rel_attr = (obj['name'], dummy_rel, attribute)
            tuple_indices = []
            if obj_rel_attr in master_ds_obj_rel_attr_imageid:
                master_ds_obj_rel_attr_imageid[obj_rel_attr].extend([idx])
            else:
                master_ds_obj_rel_attr_imageid[obj_rel_attr] = [idx]

In [30]:
# making the obj_rel_attr_imageid have unique images in its list (removing the redundancy of same tuple apprearing more than once in the same image)
for key in master_ds_obj_rel_attr_imageid:
    unique_image_list = []
    for item in master_ds_obj_rel_attr_imageid[key]:
        unique_image_list.append(item)
    unique_image_list = list(set(unique_image_list))
    master_ds_obj_rel_attr_imageid[key] = unique_image_list

In [31]:
n_items = take(1, object_vocab.items())
print(n_items)
n_items = take(1, inv_redundant_obj_dict.items())
print(n_items)

[('nose', 0)]
[('nose', '3952616')]


In [32]:
inverted_obj_rel_obj_image = {}
for key in master_ds_obj_rel_obj_imageid:
    obj1 = object_vocab[key[0]]
    obj2 = object_vocab[key[2]]
    rel = relation_vocab[key[1]]
    image_list = []
    for item in master_ds_obj_rel_obj_imageid[key]:
        image_list.append(image_vocab[item])
    inverted_obj_rel_obj_image[(obj1, rel, obj2)] = image_list


In [33]:
n_items = take(1, attr_vocab.items())
print(n_items)

[('blue', 0)]


In [34]:
inverted_obj_rel_attr_image = {}
for key in master_ds_obj_rel_attr_imageid:
    obj = object_vocab[key[0]]
    attr = attr_vocab[key[2]]
    rel = relation_vocab[key[1]]
    image_list = []
    for item in master_ds_obj_rel_attr_imageid[key]:
        image_list.append(image_vocab[item])
    inverted_obj_rel_attr_image[(obj, rel, attr)] = image_list

In [35]:
# creating object inverted index
inverted_index_object = {}

for idx, record in gqa_vg_train.items():
    for obj_key, obj in record['objects'].items():
        if obj['name'] in inverted_index_object:
            inverted_index_object[obj['name']].extend([idx])
        else:
            inverted_index_object[obj['name']] = [idx]

In [36]:
len(inverted_index_object)

422

In [37]:
n_items = take(1, inverted_index_object.items())
print(n_items)

[('nose', ['2353884', '2320168', '2320162', '961', '286099', '2346551', '2352130', '2361702', '2395326', '2317105'])]


In [38]:
# mapping inverted objects to vocab
inverted_obj_image = {}
for key in inverted_index_object:
    obj = object_vocab[key]
    image_list = []
    for item in inverted_index_object[key]:
        image_list.append(image_vocab[item])
    inverted_obj_image[obj] = list(set(image_list))
    

In [39]:
# creating object inverted index
inverted_index_attribute = {}

for idx, record in gqa_vg_train.items():
    for obj_key, obj in record['objects'].items():
        for item in obj['attributes']:
            if item in inverted_index_attribute:
                inverted_index_attribute[item].extend([idx])
            else:
                inverted_index_attribute[item] = [idx]

In [40]:
# mapping inverted attributes to vocab
inverted_attr_image = {}
for key in inverted_index_attribute:
    attr = attr_vocab[key]
    for item in inverted_index_attribute[key]:
        image_list.append(image_vocab[item])
    inverted_attr_image[attr] = list(set(image_list))

In [41]:
print("done with creation of the following inverted indexes : ")
print("inverted_obj_rel_obj_image, inverted_obj_rel_attr_image, inverted_obj_image, inverted_attr_image")

done with creation of the following inverted indexes : 
inverted_obj_rel_obj_image, inverted_obj_rel_attr_image, inverted_obj_image, inverted_attr_image


In [42]:
n_items = take(1, inverted_obj_rel_obj_image.items())
print(n_items)
n_items = take(1, inverted_obj_rel_attr_image.items())
print(n_items)
n_items = take(1, inverted_obj_image.items())
print(n_items)
n_items = take(1, inverted_attr_image.items())
print(n_items)

[((0, 0, 8), [0])]
[((2, 62, 0), [35, 87, 0])]
[(0, [0, 1, 32, 66, 5, 38, 74, 48, 87, 31])]
[(0, [0, 4, 5, 11, 20, 23, 29, 31, 32, 34, 35, 37, 39, 40, 41, 45, 49, 54, 55, 56, 57, 67, 69, 71, 73, 74, 75, 77, 81, 85, 86, 87, 89, 90, 93, 96, 97, 99])]


In [43]:
print("checking all the relevant vocabularies : ")
n_items = take(1, object_vocab.items())
print(n_items)
n_items = take(1, image_vocab.items())
print(n_items)
n_items = take(1, relation_vocab.items())
print(n_items)
n_items = take(1, attr_vocab.items())
print(n_items)
n_items = take(1, redundant_obj_dict.items())
print(n_items)

checking all the relevant vocabularies : 
[('nose', 0)]
[('2353884', 0)]
[('of', 0)]
[('blue', 0)]
[('2716708', 'nose')]


In [44]:
print("Creation of the schema graph :")

Creation of the schema graph :


In [45]:
objects_array=np.zeros(len(object_vocab))
objects_array.shape

(422,)

In [46]:
idx = 0
for key in object_vocab:
    objects_array[idx] = object_vocab[key]
    idx += 1

In [47]:
num_nodes = len(objects_array)

In [48]:
adj_mat_obj_obj=np.zeros((num_nodes,num_nodes))
#     for key in all_edges.keys() : 
#         node1=key[0]
#         node2=key[1]
#         for value in all_edges[key] : 
#             adj_mat[node1,node2,rel2ind[value]]=float(rel2ind[value])
for key in inverted_obj_rel_obj_image:
#     print(key, inverted_obj_rel_obj_image[key])
#     break
    obj1 = key[0]
    obj2 = key[2]
    rel = key[1]
    adj_mat_obj_obj[obj1, obj2] = relation_vocab[rel]

In [49]:
print(adj_mat_obj_obj.shape)

(422, 422)


In [50]:
attribute_array=np.zeros(len(attr_vocab))

attribute_array.shape

(166,)

In [51]:
idx = 0
for key in attr_vocab:
    attribute_array[idx] = attr_vocab[key]
    idx += 1

In [52]:
num_attrs = len(attr_vocab)
num_attrs

166

In [53]:
adj_mat_obj_attr=np.zeros((num_nodes,num_attrs))
#     for key in all_edges.keys() : 
#         node1=key[0]
#         node2=key[1]
#         for value in all_edges[key] : 
#             adj_mat[node1,node2,rel2ind[value]]=float(rel2ind[value])
for key in inverted_obj_rel_attr_image:
#     print(key, inverted_obj_rel_obj_image[key])
#     break
    obj = key[0]
    attr = key[2]
    rel = key[1]
    adj_mat_obj_attr[obj, attr] = relation_vocab[dummy_rel]

In [54]:
adj_mat_obj_attr.shape

(422, 166)

In [55]:
print("Creation of all Data structures for the Schema graph and inverted indexes is done..")

Creation of all Data structures for the Schema graph and inverted indexes is done..


In [56]:
schema_graph={'object_nodes' : objects_array, 
              'attr_nodes' : attribute_array, 
              'object_object_edges' : adj_mat_obj_obj, 
              'object_attr_edges' : adj_mat_obj_attr}
inverted_index={'object_nodes' : inverted_obj_image, 
                  'attr_nodes' : inverted_attr_image, 
                  'object_object_edges' : inverted_obj_rel_obj_image, 
                  'object_attr_edges' : inverted_obj_rel_attr_image}


In [57]:
args = 'train'

In [58]:
np.save('schema_graph_'+args+'.npy',schema_graph)
np.save('inverted_index_'+args+'.npy',inverted_index)
print("schema graph and inverted indexes are saved")

schema graph and inverted indexes are saved


In [59]:
# adding a <pad> attribute to the attr_vocab
dummy_rel = '<pad>'
print(len(attr_vocab))
attr_vocab['<pad>'] = len(attr_vocab) + 1
print(len(attr_vocab))

166
167


In [60]:
# saving all the important vocabularies 
np.save('image_vocab.npy',image_vocab)
np.save('object_vocab.npy',object_vocab)
np.save('relation_vocab.npy',relation_vocab)
np.save('attr_vocab.npy',attr_vocab)
np.save('redundant_obj_dict.npy',redundant_obj_dict)
print("image_vocab, object_vocab, relation_vocab, attr_vocab, redundant_obj_dict saved..")

image_vocab, object_vocab, relation_vocab, attr_vocab, redundant_obj_dict saved..
